In [22]:
import pandas as pd
import numpy as np
import re

%run preprocessor.ipynb

In [41]:
def get_keywords(df, query):
    # Create the regex pattern for the keywords
    query_pattern = r'\b(?:' + '|'.join(query) + r')\b'
    
    # Function to extract excerpts
    def extract_excerpts(script, pattern):
        excerpts = []
        for match in re.finditer(pattern, script, re.IGNORECASE):
            start = max(match.start() - 100, 0)
            end = match.end() + 100
            excerpt = script[start:end]
            excerpts.append(excerpt)
        return excerpts
    
    # Apply the function to each row and create a new column 'excerpts'
    df['excerpts'] = df['script'].apply(lambda x: extract_excerpts(x, query_pattern))
    
    # Filter rows where excerpts list is not empty
    df = df[df['excerpts'].str.len() > 0]
    
    # Add the 'query' column to indicate rows where keywords were found
    df['query'] = True
    
    return df

In [71]:
def update_all_eps(df):    
    all_episodes = pd.read_csv('./data/episodes.csv', index_col=0)
    all_episodes = all_episodes.rename(columns={'ep_num': 'episode'})
    merged_df = all_episodes.merge(df[['show_id', 'season', 'episode']], on=['show_id', 'season', 'episode'])
    merged_df['abortion'] = True

    all_episodes = all_episodes.merge(merged_df[['show_id', 'season', 'episode', 'abortion']], on=['show_id', 'season', 'episode'], how='left')
    all_episodes.fillna(False, inplace=True)

    return all_episodes